In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def create_schema(host, user, password, schema_name):
    try:
        connection = mysql.connector.connect(host=host, user=user, password=password)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {schema_name}")
            print(f"Database '{schema_name}' created or already exists.")
            cursor.close()
        connection.close()
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

def create_database(connection):
    cursor = connection.cursor()
    try:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Personal_Record (
            Age INT,
            Age_Group VARCHAR(100),
            Attrition BOOLEAN,
            Business_Travel VARCHAR(100),
            Daily_Rate INT,
            Department VARCHAR(100),
            Distance_From_Home INT,
            Education TINYINT,
            Education_Field VARCHAR(100),
            Employee_Number INT PRIMARY KEY,
            Environment_Satisfaction INT,
            Gender CHAR(1),
            Hourly_Rate INT,
            Job_Involvement TINYINT,
            Job_Level TINYINT,
            Job_Role VARCHAR(100),
            Job_Satisfaction TINYINT,
            Marital_Status VARCHAR(100),
            Monthly_Income INT,
            Monthly_Rate INT,
            Num_Companies_Worked INT,
            Over_Time VARCHAR(50),
            Percent_Salary_Hike DECIMAL(5,2),
            Performance_Rating TINYINT,
            Relationship_Satisfaction TINYINT,
            Satisfaction_Group VARCHAR(25), 
            Stock_Option_Level TINYINT,
            Total_Working_Years INT,
            Training_Times_Last_Year INT,
            Work_Life_Balance TINYINT,
            Years_At_Company INT,
            Years_Since_Last_Promotion INT,
            Years_With_Curr_Manager INT,
            Date_Birth INT,
            Remote_Work BOOLEAN,
            CONSTRAINT chk_Gender CHECK (Gender IN ('F', 'M')),
            CONSTRAINT chk_Over_Time CHECK (Over_Time IN ('True', 'False', 'Unknown')),
            CONSTRAINT chk_Percent_Salary_Hike CHECK (Percent_Salary_Hike BETWEEN 0 AND 1),
            CONSTRAINT chk_Education CHECK (Education BETWEEN 1 AND 5),
            CONSTRAINT chk_Job_Involvement CHECK (Job_Involvement BETWEEN 1 AND 5),
            CONSTRAINT chk_Job_Level CHECK (Job_Level BETWEEN 1 AND 5),
            CONSTRAINT chk_Job_Satisfaction CHECK (Job_Satisfaction BETWEEN 1 AND 5),
            CONSTRAINT chk_Marital_Status CHECK (Marital_Status IN ('Married', 'Divorced', 'Single', 'Unknown')),
            CONSTRAINT chk_Performance_Rating CHECK (Performance_Rating BETWEEN 1 AND 5),
            CONSTRAINT chk_Relationship_Satisfaction CHECK (Relationship_Satisfaction BETWEEN 1 AND 5),
            CONSTRAINT chk_Stock_Option_Level CHECK (Stock_Option_Level BETWEEN 1 AND 4),
            CONSTRAINT chk_Work_Life_Balance CHECK (Work_Life_Balance BETWEEN 1 AND 5)
        );
        """)
        connection.commit()
        print("Table created successfully.")
    except Error as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()

def load_data(df, connection):
    cursor = connection.cursor()
    cols = ', '.join([f"`{col}`" for col in df.columns])
    placeholders = ', '.join(['%s'] * len(df.columns))
    sql = f"INSERT INTO Personal_Record ({cols}) VALUES ({placeholders})"
    try:
        for index, row in df.iterrows():
            cursor.execute(sql, tuple(row))
        connection.commit()
        print(f"{cursor.rowcount} registros insertados correctamente.")
    except Error as e:
        print(f"Error al insertar datos: {e}")
        connection.rollback()
    finally:
        cursor.close()

# Main execution sequence
host = '127.0.0.1'
user = 'root'
password = 'AlumnaAdalab'
schema_name = 'ABC_Corporation'

# Create schema and connect to it
create_schema(host, user, password, schema_name)
connection = mysql.connector.connect(host=host, user=user, password=password, database=schema_name)

# Ensure the table exists
create_database(connection)

# Load data into the table
df = pd.read_csv('HR_CLEAN_DATA_2.csv')
load_data(df, connection)

# Close connection
connection.close()


Database 'ABC_Corporation' created or already exists.
Table created successfully.
Error al insertar datos: 1054 (42S22): Unknown column 'nan' in 'field list'


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('HR_CLEAN_DATA_2.csv')
nan_columns = df.columns[df.isna().any()].tolist()
print("Columns with NaN values:", nan_columns)

Columns with NaN values: ['Satisfaction_Group']
